### Задание
Используя RandomForestClassifier и еще 2 метода МО предсказать будущий ABC класс на основе любых фичей

Я буду использовать RandomForestClassifier, Gradient Boosting и Logistic Regression.

В рамках данного метода товары делят на группы по следующему принципу:\
    1. А = 20% ассортимента товаров, 80% продаж\
    2. В = 30% ассортимента товаров, 15% продаж\
    3. С = 50% товаров, 5% продаж

In [1]:
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
import pyclustering as pcl
import matplotlib.pyplot as plt

from sklearn import preprocessing
le = preprocessing.LabelEncoder()

from sklearn.metrics import accuracy_score
classes_dict = {0:"A",1: "B", 2: "C"}

def accuracy_percent(x_train, y_train, x_test, y_test, classification_model): #
    classification_model.fit(x_train, y_train)
    y_pred = classification_model.predict(x_test)
    
    print(f'{classification_model}\nScore is {classification_model.score(x_train, y_train)*100}%')
    
def class_predict(feature_value_str, classification_model, scaler): #
    data_scaled = scaler.transform(feature_value_str)
    class_ = classification_model.predict(data_scaled)
    pred_class = classes_dict[class_[0]]
    
    print(f'Predicted class: {pred_class}')
    
    return pred_class

In [2]:
FILEPATH = "./DataSet_w_NA.xlsx"

df = pd.read_excel(FILEPATH, sheet_name="Испорченные факты").dropna()
df.head()

,Номер заказа,Возврат,Дата заказа,Дата отправки,Режим отправки,Факты.Контрагент ID,Факты.Товар ID,"Продажи, руб","Продажи, шт",Скидка,Маржинальная прибыль,Повторение заказа,Повторение контрагента,Повторение товара
0,2014-100279,Нет,2015-09-10 00:00:00,2015-09-14 00:00:00,Стандарт,CLNT0000422,PROD0000870,11190.0,2.0,0.0,5371.20,1,7,3
1,2014-100293,Нет,2015-09-14 00:00:00,2015-09-18 00:00:00,Стандарт,CLNT0000784,PROD0000646,45528.0,6.0,0.2,15934.80,1,4,8
2,2014-100328,Нет,2015-07-28 00:00:00,2015-08-03 00:00:00,Стандарт,CLNT0000342,PROD0000187,1964.0,1.0,0.2,662.85,1,4,12
3,2014-100363,Нет,2015-10-08 00:00:00,2015-10-15 00:00:00,Стандарт,CLNT0000146,PROD0001095,1184.0,2.0,0.2,414.40,2,12,7
4,2014-100363,Нет,2015-10-08 00:00:00,2015-10-15 00:00:00,Стандарт,CLNT0000146,PROD0001616,9504.0,3.0,0.2,3445.20,2,12,3


Построим сводную таблицу

pivot_table:\
index -- поле, по которому будет проводиться группировка\
values -- по каким полям будут произведено аггрегирование\
aggfunc -- функции, которые необходимо применить к полям

In [3]:
df = pd.pivot_table(df, index='Факты.Товар ID', values = ['Продажи, шт', 'Продажи, руб', 'Маржинальная прибыль',
                                                             'Повторение заказа', 'Повторение товара'], 
                       aggfunc={'Продажи, шт': [np.median, np.sum],
                              'Продажи, руб': [np.median, np.sum],
                              'Повторение заказа': np.sum,
                              'Маржинальная прибыль': np.sum})

newname=df.columns.map('_'.join)
df.columns=newname
df=df.reset_index()

df

,Факты.Товар ID,Маржинальная прибыль_sum,Повторение заказа_sum,"Продажи, руб_median","Продажи, руб_sum","Продажи, шт_median","Продажи, шт_sum"
0,PROD0000001,-28160.70,6,144078.0,631978.5,2.5,12.0
1,PROD0000002,963721.00,32,396467.5,5318764.0,3.5,51.0
2,PROD0000003,32200.55,18,7310.0,76755.0,2.0,23.0
3,PROD0000004,-266380.65,17,504904.5,3621384.0,4.0,29.0
4,PROD0000005,31874.40,18,27960.0,150984.0,5.0,24.0
...,...,...,...,...,...,...,...
1857,PROD0001890,18223.65,3,161988.0,161988.0,3.0,3.0
1858,PROD0001891,-82476.90,6,112468.5,112468.5,3.0,3.0
1859,PROD0001892,27680.80,1,98860.0,98860.0,4.0,4.0
1860,PROD0001893,-187496.25,2,749985.0,749985.0,5.0,5.0


In [4]:
total_sale=df['Продажи, руб_sum'].sum()                         # посчитали общую сумму продаж
df['Доля']=df['Продажи, руб_sum']/total_sale * 100              # какой процент от общей продажи составляют продажи продукта
df = df.sort_values(by=('Продажи, руб_sum'), ascending=False)   # сортируем df по убыванию
df = df.assign(sum_d=df['Доля'].cumsum())                       # добавили в df новый столбец sum_d с куммулятивной суммой
                                                                                     # значений поля доля

df.loc[(df['sum_d'] <= 80), 'ABC'] = 'A'                        # заполним колонку ABC соответсвующими значениями
df.loc[(df['sum_d'] > 80) & (df['sum_d'] <= 95), 'ABC'] = 'B'
df.loc[(df['sum_d'] > 95), 'ABC'] = 'C'
le.fit(["A", "B", "C"])
df['class'] = le.transform(df['ABC'])

In [5]:
df

,Факты.Товар ID,Маржинальная прибыль_sum,Повторение заказа_sum,"Продажи, руб_median","Продажи, руб_sum","Продажи, шт_median","Продажи, шт_sum",Доля,sum_d,ABC,class
1388,PROD0001406,1.259996e+07,17,5599984.0,30799912.0,4.0,20.0,2.681845,2.681845,A,0
450,PROD0000454,3.876520e+06,23,1143891.0,13726692.0,3.0,31.0,1.195226,3.877071,A,0
1411,PROD0001430,-9.055392e+05,7,11319240.0,11319240.0,6.0,6.0,0.985602,4.862673,A,0
483,PROD0000488,1.746230e-10,15,1401960.0,10935288.0,5.0,39.0,0.952170,5.814843,A,0
1176,PROD0001187,1.116753e+06,33,896990.0,9911739.5,3.0,37.0,0.863046,6.677889,A,0
...,...,...,...,...,...,...,...,...,...,...,...
1838,PROD0001871,1.578500e+03,1,3850.0,3850.0,2.0,2.0,0.000335,99.999089,C,2
1854,PROD0001887,1.193400e+03,1,3536.0,3536.0,2.0,2.0,0.000308,99.999396,C,2
1747,PROD0001779,1.555200e+03,3,3240.0,3240.0,1.0,1.0,0.000282,99.999679,C,2
1675,PROD0001703,1.411200e+03,1,2880.0,2880.0,2.0,2.0,0.000251,99.999929,C,2


Этот класс будем классифицировать

In [6]:
df[df['class']==1]

,Факты.Товар ID,Маржинальная прибыль_sum,Повторение заказа_sum,"Продажи, руб_median","Продажи, руб_sum","Продажи, шт_median","Продажи, шт_sum",Доля,sum_d,ABC,class
1686,PROD0001714,-51741.00,4,342065.5,684131.0,8.0,16.0,0.059569,80.034532,B,1
82,PROD0000083,-63353.40,11,115188.0,681529.0,3.0,16.0,0.059343,80.093875,B,1
1797,PROD0001830,-9735.00,3,681450.0,681450.0,3.0,3.0,0.059336,80.153211,B,1
970,PROD0000980,279653.40,18,146965.0,680238.0,3.5,18.0,0.059230,80.212441,B,1
556,PROD0000561,257245.45,12,113305.5,679833.0,3.0,19.0,0.059195,80.271637,B,1
...,...,...,...,...,...,...,...,...,...,...,...
1484,PROD0001505,19905.60,9,82940.0,165880.0,4.0,8.0,0.014444,94.931238,B,1
369,PROD0000372,29795.15,18,23980.0,165462.0,5.0,31.0,0.014407,94.945645,B,1
1421,PROD0001441,-24291.30,31,15588.0,164973.0,3.0,31.0,0.014365,94.960010,B,1
759,PROD0000765,70479.50,10,61960.0,164194.0,4.0,11.0,0.014297,94.974307,B,1


Эти данные будем использовать

In [7]:
df[['Продажи, руб_median', 'Продажи, шт_sum', 'Доля', 'Маржинальная прибыль_sum', 'Повторение заказа_sum']]

,"Продажи, руб_median","Продажи, шт_sum",Доля,Маржинальная прибыль_sum,Повторение заказа_sum
1388,5599984.0,20.0,2.681845,1.259996e+07,17
450,1143891.0,31.0,1.195226,3.876520e+06,23
1411,11319240.0,6.0,0.985602,-9.055392e+05,7
483,1401960.0,39.0,0.952170,1.746230e-10,15
1176,896990.0,37.0,0.863046,1.116753e+06,33
...,...,...,...,...,...
1838,3850.0,2.0,0.000335,1.578500e+03,1
1854,3536.0,2.0,0.000308,1.193400e+03,1
1747,3240.0,1.0,0.000282,1.555200e+03,3
1675,2880.0,2.0,0.000251,1.411200e+03,1


In [11]:
scaler = StandardScaler()

XColumns = ['Продажи, руб_median', 'Продажи, шт_sum', 'Доля', 'Маржинальная прибыль_sum', 'Повторение заказа_sum']
YColumns = ['class']

dfX = df[XColumns]
dfY = df[YColumns]
X_train, X_test, y_train, y_test = train_test_split(dfX, dfY, test_size=0.20, random_state = 1)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
    
data = [[25000, 10, 0.05,20000, 10]] # взяли типичного представителя класса B

methods = [  LogisticRegression(max_iter=10000)
           , RandomForestClassifier()
           , GradientBoostingClassifier()   ]

for method in methods:
    accuracy_percent(X_train, y_train, X_test, y_test, method)
    class_predict(data, rfc_model, scaler)
    print('\n')

/home/liza/env/env/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/liza/env/env/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/tmp/ipykernel_414/748459086.py:22: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classification_model.fit(x_train, y_train)


LogisticRegression(max_iter=10000)
Score is 94.6272666218939%
Predicted class: B


RandomForestClassifier()
Score is 100.0%
Predicted class: B




/home/liza/env/env/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/liza/env/env/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifier()
Score is 100.0%
Predicted class: B




/home/liza/env/env/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


### Вывод
класс объекта можно предсказывать, лучше всего справились ансамблевые методы